# New Section

In [44]:

import sys
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import sklearn.model_selection
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

import datetime

import joblib
import pickle
import tempfile


import boto3
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()


# S3 bucket name
bucket = 'sagemaker-us-west-2-479862333671'
subfolder = ''
conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']


In [45]:
model_data='s3://sagemaker-us-west-2-479862333671/perdictions/output/sagemaker-xgboost-2022-01-10-08-59-02-189/output/model.tar.gz'

In [46]:
model_data

's3://sagemaker-us-west-2-479862333671/perdictions/output/sagemaker-xgboost-2022-01-10-08-59-02-189/output/model.tar.gz'

In [47]:
from sagemaker import image_uris
image_uris.retrieve(framework='xgboost',region='us-west-2',version='1.2-1')

'246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.2-1'

In [48]:
image=image_uris.retrieve(framework='xgboost',region='us-west-2',version='1.2-1')

In [49]:
image

'246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.2-1'

In [50]:
role

'arn:aws:iam::479862333671:role/service-role/AmazonSageMaker-ExecutionRole-20220103T082598'

In [51]:
val = pd.read_csv('s3://{}/{}'.format(bucket, 'data/val.csv'))

In [52]:
val

,demand,item_id,dept_id,cat_id,store_id,state_id,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,date_day,date_week,date_month,date_year,date_week_m,lag_28,lag_35,lag_42,lag_49,lag_56,rolling_median_7,rolling_median_14,rolling_median_30,rolling_median_60,rolling_median_180
0,0,1437,3,1,0,0,1914,30,4,4,2,0,0,0,8.38,25,17,4,5,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1437,3,1,0,0,1915,30,4,4,2,0,0,0,8.38,26,17,4,5,4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0,1437,3,1,0,0,1916,30,4,4,2,0,0,0,8.38,27,17,4,5,4,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2,1437,3,1,0,0,1917,30,4,4,2,0,0,0,8.38,28,17,4,5,4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0,1437,3,1,0,0,1918,30,4,4,2,0,0,0,8.38,29,17,4,5,5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823225,1,1432,2,0,9,2,1940,30,4,4,2,0,0,0,2.98,21,20,5,5,3,0.0,0.0,0.0,0.0,0.0,1.0,0.5,1.0,1.0,1.0
823226,1,1433,2,0,9,2,1940,30,4,4,2,0,0,0,2.48,21,20,5,5,3,3.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
823227,0,1434,2,0,9,2,1940,30,4,4,2,0,0,0,3.98,21,20,5,5,3,5.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0
823228,1,1435,2,0,9,2,1940,30,4,4,2,0,0,0,1.28,21,20,5,5,3,0.0,0.0,3.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0


In [10]:
trainedmodel = sagemaker.model.Model(
    image_uri=image,
    model_data = model_data,
    role=role)

In [11]:
model_perdictor = trainedmodel.deploy(initial_instance_count=1, instance_type='ml.c5.4xlarge')

------!

In [53]:

predictor = sagemaker.predictor.RealTimePredictor(
    endpoint_name='sagemaker-xgboost-2022-01-10-11-29-40-077',
    content_type='text/csv')

The class RealTimePredictor has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
content_type is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [54]:
val.iloc[:,1:]

,item_id,dept_id,cat_id,store_id,state_id,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,date_day,date_week,date_month,date_year,date_week_m,lag_28,lag_35,lag_42,lag_49,lag_56,rolling_median_7,rolling_median_14,rolling_median_30,rolling_median_60,rolling_median_180
0,1437,3,1,0,0,1914,30,4,4,2,0,0,0,8.38,25,17,4,5,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1437,3,1,0,0,1915,30,4,4,2,0,0,0,8.38,26,17,4,5,4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1437,3,1,0,0,1916,30,4,4,2,0,0,0,8.38,27,17,4,5,4,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1437,3,1,0,0,1917,30,4,4,2,0,0,0,8.38,28,17,4,5,4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1437,3,1,0,0,1918,30,4,4,2,0,0,0,8.38,29,17,4,5,5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823225,1432,2,0,9,2,1940,30,4,4,2,0,0,0,2.98,21,20,5,5,3,0.0,0.0,0.0,0.0,0.0,1.0,0.5,1.0,1.0,1.0
823226,1433,2,0,9,2,1940,30,4,4,2,0,0,0,2.48,21,20,5,5,3,3.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
823227,1434,2,0,9,2,1940,30,4,4,2,0,0,0,3.98,21,20,5,5,3,5.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0
823228,1435,2,0,9,2,1940,30,4,4,2,0,0,0,1.28,21,20,5,5,3,0.0,0.0,3.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0


In [64]:
val.iloc[:,1:].to_csv('X_val.csv')

In [65]:
data_dir = ''

val.iloc[:,1:].to_csv(os.path.join(data_dir, 'X_val.csv'), index=False)
X_val_location = sagemaker_session.upload_data(os.path.join(data_dir, 'X_val.csv'))


In [66]:
predictions = predictor.predict('s3://{}/{}'.format(bucket, 'data/X_val.csv')).decode('utf-8')

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (415) from primary with message "application/octet-stream is not an accepted ContentType: csv, libsvm, parquet, recordio-protobuf, text/csv, text/libsvm, text/x-libsvm, application/x-parquet, application/x-recordio-protobuf.". See https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logEventViewer:group=/aws/sagemaker/Endpoints/sagemaker-xgboost-2022-01-10-11-29-40-077 in account 479862333671 for more information.

In [ ]:
predictions